In [1]:
import pandas as pd
import jieba
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import imageio
from collections import OrderedDict
from pyecharts.charts import Bar

In [11]:
#import data
March_filtered = pd.read_csv('27MarTopTotalSearch.csv', names=['content','end_time','start_time','volume','relevancy'])
March_filtered=March_filtered[1:]
content=March_filtered['content']
March_filtered.head()

,content,end_time,start_time,volume,relevancy
1,肖战工作室道歉,2/3/2020 14:00,1/3/2020 21:48,8580306,0
2,郑恺苗苗,10/3/2020 20:22,10/3/2020 10:42,6104855,0
3,唐艺昕肚子,16/3/2020 19:30,16/3/2020 11:26,5821712,0
4,张檬金恩圣公布恋情,20/3/2020 5:44,19/3/2020 10:06,5711930,0
5,刘真去世,23/3/2020 16:58,23/3/2020 10:44,5553134,0


In [12]:
# combine content of cells into text string
content_text = ''
for i in content:
    content_text += ''.join(i)


In [13]:
# tokenlization
words = jieba.lcut(content_text) 
word_string = " ".join(words)

#clean data with stop words
stop_words = jieba.load_userdict("stop_words") # load the file
stopwords=pd.read_csv('stop_words',index_col=False,quoting=3,sep="\t",names=['stopword'], encoding='utf-8')
stopwords=stopwords['stopword'].values
stopwords = stopwords.tolist()
# stopwords += ['新型','肺炎','一家','一架']  #manual update stop words


Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Zhengnan\AppData\Local\Temp\jieba.cache
Loading model cost 0.727 seconds.
Prefix dict has been built succesfully.


In [22]:
# get words frequency for most frequent 10 words
counts = {}
for word in words:
    if word not in stopwords and len(word)>1:
        if len(word) == 1:    
            continue
        else:
            counts[word] = counts.get(word, 0) + 1   

ordered_counts = OrderedDict(sorted(counts.items(), key=lambda item: item[1],reverse=True)[:11])


# seperate word with its frequency
key_list = [key for key in ordered_counts] #words_list
value_list = []                            #frequency list
for key in key_list:
    value_list.append(ordered_counts[key])


In [27]:
from pyecharts import options as opts
#create bar chart for words frequency
bar = (
    Bar()
    .add_xaxis(key_list)
    .add_yaxis("frequency", value_list, color= "#5793f3")
    .set_global_opts(title_opts=opts.TitleOpts(title='Frequency of top search words For March'))
)
bar.render("Mar_topSearch.html") #rename it

# Create Word Cloud
mk = imageio.imread("chinamap.png")

w = WordCloud(width=1000,
                        height=700,
                        background_color='white',
                        font_path='msyh.ttc',
                        mask=mk,
                        stopwords={'新冠','肺炎','冠状病毒','病例','武汉','确诊'},
                        scale=15)

w.generate(word_string)
w.to_file('March_filtered_wordcloud.png') #rename it